<a href="https://colab.research.google.com/github/eceirem/Hukuki-Asistan/blob/hibrit_RRF_Cross-Encoder_BM25/hibrit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uygulama ve Arama Motoru (Demonstrasyon)

**Amacı: **Bir ölçüm yapmaktan ziyade, sistemin canlı bir arama motoru gibi nasıl çalıştığını göstermek. İlk dökümanın özetini alır ve ona en yakın 5 emsali bulup ekrana yazdırır.

**Kullanılan Yöntemler: **Segment ayıklama (extract_sections), RRF Fusion ve Cross-Encoder Reranking.

In [ ]:
# ===========================================================
# HUKUKİ ASİSTAN - GELİŞMİŞ HİBRİT EMSAL ARAMA MOTORU (v2)
# Mimari: BM25 + Bi-Encoder -> RRF Fusion -> Cross-Encoder Rerank
# ===========================================================

import os, glob, json, re, random
import numpy as np
import torch
from tqdm import tqdm
from typing import Dict, Any, List, Tuple, Optional


!pip install -q rank-bm25 sentence-transformers tqdm

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. CONFIG & PATHS ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements" # Google Drive yoluna göre ayarlı [cite: 43]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # [cite: 123, 208]
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2" # Yeniden sıralama için usta dokunuşu

# --- 2. VERİ YÜKLEME VE TEMİZLEME ---
def clean_text(s: Any) -> str:
    s = "" if s is None else str(s)
    return re.sub(r"\s+", " ", s).strip()

def load_legal_corpus(data_dir: str):
    paths = sorted(glob.glob(os.path.join(data_dir, "*.json")))
    raw_objs, doc_ids = [], []
    for p in tqdm(paths, desc="JSON Kararlar Yükleniyor"):
        with open(p, "r", encoding="utf-8") as f:
            raw_objs.append(json.load(f))
        doc_ids.append(os.path.basename(p))
    return raw_objs, doc_ids

# Karar bölümlerini makaleye uygun şekilde ayıklıyoruz [cite: 38, 99]
def extract_sections(obj: Dict[str, Any]):
    rrl = obj.get("rrl_segments") or {}
    sf  = obj.get("structural_features") or {}
    md  = obj.get("meta_data") or {}

    return {
        "summary": clean_text(obj.get("summary_for_model")), # Sorgu olarak kullanılacak [cite: 126]
        "facts": clean_text(rrl.get("facts_text")),
        "reasoning": clean_text(rrl.get("reasoning_text")), # En ayırt edici bölüm [cite: 15, 190]
        "laws": " ".join([clean_text(x) for x in sf.get("mentioned_laws", [])]),
        "full_index": f"{rrl.get('facts_text', '')} {rrl.get('reasoning_text', '')} {sf.get('mentioned_laws', '')}".strip()
    }

# --- 3. ARAMA VE SIRALAMA ALGORİTMALARI ---

# RRF: İki farklı modelin sıralamasını adil şekilde birleştirir
def rrf_fusion(ranks_list: List[np.ndarray], k: int = 60):
    rrf_scores = np.zeros(ranks_list[0].shape)
    for ranks in ranks_list:
        rrf_scores += 1.0 / (k + ranks + 1)
    return rrf_scores

# Cross-Encoder: İlk 50 adayı derinlemesine analiz eder
def apply_reranking(ce_model, query, candidates, top_k=10):
    pairs = [[query, cand] for cand in candidates]
    scores = ce_model.predict(pairs)
    return np.argsort(-scores)[:top_k]

# --- 4. ANA ÇALIŞTIRICI (MAIN PIPELINE) ---

def main():
    # Ensure Google Drive is mounted
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
    except ImportError:
        print("Google Colab environment not detected. Skipping drive mount.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")

    # 1. Veriyi Oku
    raw_objs, doc_ids = load_legal_corpus(DATA_DIR)

    if not raw_objs:
        print(f"UYARI: '{DATA_DIR}' dizininden hiç hukuki karar dosyası yüklenemedi. Lütfen yolun doğru olduğundan ve JSON dosyalarının mevcut olduğundan emin olun.")
        return # Exit if no data is loaded

    processed_data = [extract_sections(o) for o in raw_objs]

    # Filter out entries where 'reasoning' is empty, as this is used for corpus_texts
    filtered_data = [d for i, d in enumerate(processed_data) if d["reasoning"] and doc_ids[i] is not None]
    filtered_doc_ids = [doc_ids[i] for i, d in enumerate(processed_data) if d["reasoning"] and doc_ids[i] is not None]

    if not filtered_data:
        print("UYARI: 'Gerekçe' (Reasoning) bölümü içeren geçerli metin bulunamadı. BM25 ve Bi-Encoder indekslemesi yapılamıyor.")
        return # Exit if no valid data for corpus

    # Boş olmayan sorguları ve dökümanları ayır [cite: 101]
    # 'summary' alanı boş olmayanları alıyoruz
    queries = [d["summary"] for d in filtered_data if d["summary"]]

    if not queries:
        print("UYARI: Sorgu için kullanılabilecek geçerli 'summary' alanı bulunamadı. Test sorgusu oluşturulamıyor.")
        return # Exit if no queries can be formed

    corpus_texts = [d["reasoning"] for d in filtered_data]

    # 2. BM25 Hazırla (Kelimeden yakalar) [cite: 118, 119]
    tokenized_corpus = [re.findall(r"\w+", text.lower()) for text in corpus_texts]
    bm25 = BM25Okapi(tokenized_corpus)

    # 3. Bi-Encoder Hazırla (Anlamdan yakalar) [cite: 122, 210]
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)
    doc_embeddings = bi_model.encode(corpus_texts, convert_to_numpy=True, show_progress_bar=True)
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)

    # 4. Örnek Bir Sorgu Üzerinde Test Et
    test_query = queries[0]
    print(f"\nSorgu (Summary for Model): {test_query[:150]}...\n")

    # A) BM25 Skorları ve Sıralaması
    bm25_scores = bm25.get_scores(re.findall(r"\w+", test_query.lower()))
    bm25_ranks = np.argsort(np.argsort(-bm25_scores))

    # B) Dense (Bi-Encoder) Skorları ve Sıralaması
    query_emb = bi_model.encode(test_query)
    dense_scores = query_emb @ doc_embeddings.T
    dense_ranks = np.argsort(np.argsort(-dense_scores))

    # C) RRF Hibrit Birleştirme (Yenilik!) [cite: 141, 194]
    combined_scores = rrf_fusion([bm25_ranks, dense_ranks])
    top_50_indices = np.argsort(-combined_scores)[:50]

    # D) Cross-Encoder Reranking (En Üst Düzey Hassasiyet)
    candidate_texts = [corpus_texts[idx] for idx in top_50_indices]
    reranked_sub_indices = apply_reranking(ce_model, test_query, candidate_texts, top_k=5)
    final_indices = top_50_indices[reranked_sub_indices]

    # 5. Sonuçları Yazdır
    print("--- EN YAKIN EMSAL KARARLAR ---")
    for i, idx in enumerate(final_indices):
        print(f"{i+1}. Dosya: {filtered_doc_ids[idx]} | Skor (RRF): {combined_scores[idx]:.4f}")
        print(f"   Gerekçe Özeti: {corpus_texts[idx][:200]}...\n")

if __name__ == "__main__":
    main()

# Performans Ölçüm Paneli (Hızlı Benchmark)

**Amacı: **Kod 1 ile neredeyse aynıdır ancak veri yükleme ve işleme kısımları daha sadedir. Kod 1'in daha doğrudan ve hızlı çalışan bir versiyonudur.

**Kullanılan Yöntemler: **BM25, Bi-Encoder ve Cross-Encoder.

In [ ]:
import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm # Döngüleri izlemek için usta dokunuşu
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. AYARLAR VE YOLLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# --- 2. EVALUATION FONKSİYONU ---
def calculate_recall_at_k(ranks, k_values=[1, 5, 10]):
    results = {}
    for k in k_values:
        hits = sum(1 for r in ranks if r < k)
        results[f"R@{k}"] = round(hits / len(ranks), 4)
    return results

# --- 3. ANA BENCHMARK FONKSİYONU ---
def run_benchmark():
    # A) Dosya Yükleme
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    corpus_data = []

    for p in tqdm(paths, desc="1. Dosyalar Diskten Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            rrl = obj.get("rrl_segments", {})
            corpus_data.append({
                "id": os.path.basename(p),
                "query": obj.get("summary_for_model", ""),
                "reasoning": rrl.get("reasoning_text", ""),
            })

    # B) Filtreleme
    valid_data = [d for d in corpus_data if d["query"] and d["reasoning"]]
    queries = [d["query"] for d in valid_data]
    docs = [d["reasoning"] for d in valid_data]
    total = len(queries)

    print(f"\n✅ Veri Hazır: {total} geçerli döküman/sorgu bulundu.\n")

    # --- MODEL 1: BM25 (Sparse) ---
    print("🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...")
    tokenized_corpus = [re.findall(r"\w+", d.lower()) for d in docs]
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_ranks = []
    for i, q in tqdm(enumerate(queries), total=total, desc="   BM25 Sıralanıyor"):
        scores = bm25.get_scores(re.findall(r"\w+", q.lower()))
        rank = np.argsort(-scores).tolist().index(i)
        bm25_ranks.append(rank)

    results_bm25 = calculate_recall_at_k(bm25_ranks)
    results_bm25["Model"] = "BM25 (Reasoning)"

    # --- MODEL 2: BI-ENCODER (Dense) ---
    print("\n🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...")
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)

    # Encoding aşamasını izle
    print("   Vektörler oluşturuluyor (Bu aşama GPU hızına bağlıdır)...")
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)

    bi_ranks = []
    for i in tqdm(range(total), desc="   Vektörler Karşılaştırılıyor"):
        scores = query_embs[i] @ doc_embs.T
        rank = np.argsort(-scores).tolist().index(i)
        bi_ranks.append(rank)

    results_bi = calculate_recall_at_k(bi_ranks)
    results_bi["Model"] = "Dense (MPNet)"

    # --- MODEL 3: HİBRİT + CROSS-ENCODER (Usta İşi) ---
    print("\n🚀 Adım 3: Hibrit + Cross-Encoder Re-rank Başlıyor...")
    print("   ⚠️ UYARI: Bu aşama en ağır olanıdır, derinlemesine analiz yapar.")
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
    final_ranks = []
    K = 60

    # Ana döngüyü izle
    for i in tqdm(range(total), desc="   Derin Analiz (Re-ranking)"):
        # 1. BM25 & Dense skorlarını al
        b_scores = bm25.get_scores(re.findall(r"\w+", queries[i].lower()))
        b_rank_indices = np.argsort(-b_scores)
        d_scores = query_embs[i] @ doc_embs.T
        d_rank_indices = np.argsort(-d_scores)

        # 2. RRF Fusion
        rrf_scores = np.zeros(total)
        for rank, idx in enumerate(b_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)
        for rank, idx in enumerate(d_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)

        # 3. İlk 20 adayı Cross-Encoder'a gönder
        top_20_candidates = np.argsort(-rrf_scores)[:20]
        pairs = [[queries[i], docs[idx]] for idx in top_20_candidates]
        ce_scores = ce_model.predict(pairs, show_progress_bar=False) # Her adımda iç bar gerekmez

        # 4. Final Sıralama
        best_within_top20 = np.argsort(-ce_scores)
        final_top_indices = top_20_candidates[best_within_top20]

        if i in final_top_indices:
            rank = final_top_indices.tolist().index(i)
        else:
            rank = 999
        final_ranks.append(rank)

    results_hybrid = calculate_recall_at_k(final_ranks)
    results_hybrid["Model"] = "Bizim Hibrit (RRF + CE)"

    # --- SONUÇLARI TABLOLAŞTIR ---
    df = pd.DataFrame([results_bm25, results_bi, results_hybrid])
    print("\n" + "="*50)
    print("📊 TEST SONUÇLARI TAMAMLANDI")
    print("="*50)
    return df[["Model", "R@1", "R@5", "R@10"]]

# --- ÇALIŞTIR ---
comparison_table = run_benchmark()
print(comparison_table)

# Geliştirilmiş Benchmark Kodu

**Amacı:** Sistemdeki tüm modelleri (BM25, Dense ve Hibrit) veri setinin tamamında test edip "Hangi model daha başarılı?" sorusuna bilimsel (Recall@K) yanıt vermek.

**Kullanılan Yöntemler:** BM25 (Kelime bazlı), MPNet (Anlamsal/Vektörel), RRF (Sıralama Birleştirme) ve Cross-Encoder (Yeniden Sıralama).

**Öne Çıkan Özelliği:** Hata payını minimize eden temizlik fonksiyonları (clean_text) ve dökümanların listede bulunamaması durumuna karşı korumalı bir yapı içerir.

In [ ]:
import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm # Döngüleri izlemek için usta dokunuşu
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
from typing import Any
from google.colab import drive # Import drive for mounting

# --- 1. AYARLAR VE YOLLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# --- Yardımcı Fonksiyon: Metin Temizleme ---
def clean_text(s: Any) -> str:
    s = "" if s is None else str(s)
    return re.sub(r"\s+", " ", s).strip()

# --- 2. EVALUATION FONKSİYONU ---
def calculate_recall_at_k(ranks, k_values=[1, 5, 10]):
    results = {}
    for k in k_values:
        hits = sum(1 for r in ranks if r < k)
        results[f"R@{k}"] = round(hits / len(ranks), 4)
    return results

# --- 3. ANA BENCHMARK FONKSİYONU ---
def run_benchmark():
    # Ensure Google Drive is mounted
    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"]) # Exit if mount fails

    # A) Dosya Yükleme
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    corpus_data = []

    if not paths:
        print(f"UYARI: '{DATA_DIR}' dizininde hiç hukuki karar dosyası bulunamadı.")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"])

    for p in tqdm(paths, desc="1. Dosyalar Diskten Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            rrl = obj.get("rrl_segments", {})
            corpus_data.append({
                "id": os.path.basename(p),
                "query": clean_text(obj.get("summary_for_model", "")),
                "reasoning": clean_text(rrl.get("reasoning_text", "")),
            })

    # B) Filtreleme
    valid_data = [d for d in corpus_data if d["query"] and d["reasoning"]]
    queries = [d["query"] for d in valid_data]
    docs = [d["reasoning"] for d in valid_data]
    total = len(queries)

    print(f"\n✅ Veri Hazır: {total} geçerli döküman/sorgu bulundu.\n")

    if total == 0:
        print("UYARI: Hiç geçerli sorgu veya doküman bulunamadı. Benchmark çalıştırılamıyor.")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"]) # Boş DataFrame döndür

    # --- MODEL 1: BM25 (Sparse) ---
    print("🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...")
    tokenized_corpus = [re.findall(r"\w+", d.lower()) for d in docs]
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_ranks = []
    for i, q in tqdm(enumerate(queries), total=total, desc="   BM25 Sıralanıyor"):
        tokenized_query = re.findall(r"\w+", q.lower())
        if not tokenized_query: # Boş sorguları ele al
            bm25_ranks.append(999) # Çok düşük bir sıralama ver
            continue
        scores = bm25.get_scores(tokenized_query)
        rank_list = np.argsort(-scores).tolist()
        # Eğer mevcut döküman (i) sıralama listesinde değilse veya BM25 sonucu anlamsızsa
        # Güvenli erişim için kontrol ekle: i'nin rank_list içinde olduğundan emin ol
        if i < len(rank_list) and i in rank_list:
            rank = rank_list.index(i)
        else:
            rank = 999
        bm25_ranks.append(rank)

    results_bm25 = calculate_recall_at_k(bm25_ranks)
    results_bm25["Model"] = "BM25 (Reasoning)"

    # --- MODEL 2: BI-ENCODER (Dense) ---
    print("\n🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...")
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)

    # Encoding aşamasını izle
    print("   Vektörler oluşturuluyor (Bu aşama GPU hızına bağlıdır)...")
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)

    bi_ranks = []
    for i in tqdm(range(total), desc="   Vektörler Karşılaştırılıyor"):
        scores = query_embs[i] @ doc_embs.T
        rank = np.argsort(-scores).tolist().index(i)
        bi_ranks.append(rank)

    results_bi = calculate_recall_at_k(bi_ranks)
    results_bi["Model"] = "Dense (MPNet)"

    # --- MODEL 3: HİBRİT + CROSS-ENCODER (Usta İşi) ---
    print("\n🚀 Adım 3: Hibrit + Cross-Encoder Re-rank Başlıyor...")
    print("   ⚠️ UYARI: Bu aşama en ağır olanıdır, derinlemesine analiz yapar.")
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
    final_ranks = []
    K = 60

    # Ana döngüyü izle
    for i in tqdm(range(total), desc="   Derin Analiz (Re-ranking)"):
        # 1. BM25 & Dense skorlarını al
        b_scores = bm25.get_scores(re.findall(r"\w+", queries[i].lower()))
        b_rank_indices = np.argsort(-b_scores)
        d_scores = query_embs[i] @ doc_embs.T
        d_rank_indices = np.argsort(-d_scores)

        # 2. RRF Fusion
        rrf_scores = np.zeros(total)
        # Boş BM25 sorgularını ele al, aksi takdirde rrf_scores'a katkı sağlamaz
        if re.findall(r"\w+", queries[i].lower()): # Yalnızca token'lar varsa katkı sağla
            for rank, idx in enumerate(b_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)
        for rank, idx in enumerate(d_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)

        # 3. İlk 20 adayı Cross-Encoder'a gönder
        top_20_candidates = np.argsort(-rrf_scores)[:20]
        pairs = [[queries[i], docs[idx]] for idx in top_20_candidates]
        ce_scores = ce_model.predict(pairs, show_progress_bar=False) # Her adımda iç bar gerekmez

        # 4. Final Sıralama
        best_within_top20 = np.argsort(-ce_scores)
        final_top_indices = top_20_candidates[best_within_top20]

        if i in final_top_indices:
            rank = final_top_indices.tolist().index(i)
        else:
            rank = 999
        final_ranks.append(rank)

    results_hybrid = calculate_recall_at_k(final_ranks)
    results_hybrid["Model"] = "Bizim Hibrit (RRF + CE)"

    # --- SONUÇLARI TABLOLAŞTIR ---
    df = pd.DataFrame([results_bm25, results_bi, results_hybrid])
    print("\n" + "="*50)
    print("📊 TEST SONUÇLARI TAMAMLANDI")
    print("="*50)
    return df[["Model", "R@1", "R@5", "R@10"]]

# --- ÇALIŞTIR ---
comparison_table = run_benchmark()
print(comparison_table)

# ULTIMATE BENCHMARK (PARM + CROSS-ENCODER)

**Amacı:** Uzun hukuk metinlerindeki "anlam seyrelmesi" sorununu aşmak için dökümanları parçalara bölerek (chunking) en hassas ve derinlemesine emsal tespitini gerçekleştirir.

**Kullanılan Yöntemler:** Paragraph Chunking, PARM (MaxSim Pooling) ve Cross-Encoder Reranking.

In [ ]:
# ============================================================
# HUKUKİ ASİSTAN - ULTIMATE BENCHMARK (PARM + CROSS-ENCODER)
# Mimari: BM25, Bi-Encoder, PARM (Chunking) ve Cross-Encoder
# ============================================================

import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. AYARLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BI_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CE_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# 2. CHUNKING FONKSİYONU (PARM'ın Kalbi)
def get_chunks(text, chunk_size=250):
    """Metni paragraf benzeri küçük parçalara böler."""
    words = text.split()
    if not words: return [""]
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def run_ultimate_benchmark():
    # A) VERİ YÜKLEME
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    all_data = []

    for p in tqdm(paths, desc="1. Dosyalar Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            reasoning = obj.get("rrl_segments", {}).get("reasoning_text", "")
            summary = obj.get("summary_for_model", "")
            if summary and reasoning:
                all_data.append({
                    "id": os.path.basename(p),
                    "query": summary,
                    "doc": reasoning,
                    "chunks": get_chunks(reasoning) # PARM için dökümanı böldük
                })

    total = len(all_data)
    queries = [d["query"] for d in all_data]
    docs = [d["doc"] for d in all_data]

    # MODELLERİ YÜKLE
    bi_model = SentenceTransformer(BI_MODEL, device=DEVICE)
    ce_model = CrossEncoder(CE_MODEL, device=DEVICE)

    # B) VEKTÖRLEŞTİRME
    print("\n🚀 Adım 2: Vektörler Oluşturuluyor...")
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)

    # --- MODEL: PARM (İNOVATİF) ---
    print("\n🚀 Adım 3: PARM (Paragraph Aggregation) Ölçülüyor...")
    parm_ranks = []
    for i in tqdm(range(total), desc="   PARM Analizi"):
        q_emb = query_embs[i]
        doc_max_scores = []
        for d_entry in all_data:
            # Dökümanın her bir chunk'ını sorguyla kıyasla
            chunk_embs = bi_model.encode(d_entry["chunks"], convert_to_numpy=True, show_progress_bar=False)
            chunk_scores = q_emb @ chunk_embs.T
            doc_max_scores.append(np.max(chunk_scores)) # MAX POOLING: En iyi paragrafın puanını al
        parm_ranks.append(np.argsort(-np.array(doc_max_scores)).tolist().index(i))

    # --- MODEL: HİBRİT + CROSS-ENCODER ---
    print("\n🚀 Adım 4: Hibrit + Cross-Encoder Ölçülüyor...")
    final_ranks = []
    for i in tqdm(range(total), desc="   Re-ranking Analizi"):
        initial_scores = query_embs[i] @ doc_embs.T
        top_20_indices = np.argsort(-initial_scores)[:20]

        candidates = [docs[idx] for idx in top_20_indices]
        pairs = [[queries[i], cand[:1000]] for cand in candidates] # İlk 1000 karakter (Hız için)
        ce_scores = ce_model.predict(pairs, show_progress_bar=False)

        reranked_indices = top_20_indices[np.argsort(-ce_scores)]
        final_ranks.append(reranked_indices.tolist().index(i) if i in reranked_indices else 999)

    # SONUÇLARI TABLOLAŞTIR
    def get_metrics(ranks, name):
        return {"Model": name,
                "R@1": round(sum(1 for r in ranks if r < 1)/len(ranks), 4),
                "R@5": round(sum(1 for r in ranks if r < 5)/len(ranks), 4),
                "R@10": round(sum(1 for r in ranks if r < 10)/len(ranks), 4)}

    df = pd.DataFrame([
        get_metrics(parm_ranks, "PARM (Paragraph Aggregation)"),
        get_metrics(final_ranks, "Hibrit + Cross-Encoder (CE)")
    ])
    return df

# ÇALIŞTIR
results = run_ultimate_benchmark()
print(results)